# Team 4,5,6 - HW6 - ETL Data Mart HW
Data Warehousing - Fall 21

* Team 5: Vince Purcell, Brandon Mondile
* Team 4: James Law, Kevin Malone
* Team 6: Zachary Zampino, David Luco

In [7]:
import csv
import sqlite3 as lite
import datetime
from datetime import date

In [8]:
con = lite.connect(r'Grocery.db')  
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS ProductDimension")
s = 'CREATE TABLE ProductDimension(ProductKey INT, SKU varchar(8), BasePrice FLOAT, ProductName varchar(50), ProductClassID INT,'
s = s+' Subcategory varchar(50), Category varchar(50), Department varchar(50), ProductFamily varchar(50), Size varchar(10),' 
s = s+'PerCase INT, Manufacturer varchar(50), Supplier varchar(16))'
cur.execute(s)

perCase = 12
with open('ConformedProducts.txt') as csv_file:                                                                                          
    csv_reader = csv.reader(csv_file, delimiter='\t')

    count = 0
    for row in csv_reader:
        if count != 0:
            i = row[0]
            sku = row[1]
            productName = row[2]
            classId = row[3]
            subcat = row[4]
            category = row[5]
            department = row[6]
            family = row[7]
            size = row[8]
            manufacturer = row[9]
            supplier = row[10]
            basePrice = 0

            cur.execute("INSERT INTO ProductDimension VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)",
            (i, sku, basePrice, productName, classId, subcat, category, department, family, size, perCase, manufacturer, supplier))
        count += 1

con.commit()

i = 0
with open('products_first_working.csv', 'r') as csvfile:
    for row in csv.DictReader(csvfile):
        i = i + 1
        basePrice = row['BasePrice']
        cur.execute("UPDATE ProductDimension SET BasePrice = ? WHERE ProductKey = ? ",(basePrice, i))

con.commit()
con.close()

In [9]:
TEAM4_CSV = 'team4_6mo.csv'
TEAM5_CSV = 'Team5_6Mo_Transactions.csv'
TEAM6_CSV = 'extract5.csv'

INSERT_TEAM4 = 'INSERT INTO team4_6mo_transactions VALUES (?, ?, ?, ?, ?, ?);'
INSERT_TEAM5 = 'INSERT INTO team5_6mo_transactions VALUES (?, ?, ?, ?, ?, ?);'
INSERT_TEAM6 = 'INSERT INTO team6_6mo_transactions VALUES (?, ?, ?, ?, ?, ?);'

TEAM_DICT = {TEAM4_CSV: INSERT_TEAM4,
            TEAM5_CSV: INSERT_TEAM5,
            TEAM6_CSV: INSERT_TEAM6}

In [10]:
con = lite.connect(r'Grocery.db')  
cur = con.cursor()
cur.execute('drop table if exists team4_6mo_transactions')
cur.execute('create Table team4_6mo_transactions(date1 TEXT, customer_id TEXT, sku TEXT, sale_price TEXT, items_left INTEGER, cases_ordered INTEGER)')

cur.execute('drop table if exists team5_6mo_transactions')
cur.execute('create Table team5_6mo_transactions(date1 TEXT, customer_id TEXT, sku TEXT, sale_price TEXT, items_left INTEGER, cases_ordered INTEGER)')

cur.execute('drop table if exists team6_6mo_transactions')
cur.execute('create Table team6_6mo_transactions(date1 TEXT, customer_id TEXT, sku TEXT, sale_price TEXT, items_left INTEGER, cases_ordered INTEGER)')


In [11]:
for key, value in TEAM_DICT.items():
    with open(key) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        header = next(csv_reader)
        for row in csv_reader:
            row[3] = float(str(row[3]).strip('$'))
            cur.execute(value, row[0:7])
    con.commit()
con.close()

### Adding 6 months to DateDimension

In [12]:
# Searched '2020 Holidays' on Google and these were what was listed
holidays = {
    date(2021, 1, 1)    : 'New Years Day',
    date(2021, 1, 18)   : 'MLK Day',
    date(2021, 2, 15)   : 'Washingtons Birthday',
    date(2021, 4, 2)    : 'Good Friday',
    date(2021, 5, 31)   : 'Memorial Day',
    date(2021, 7, 5)    : 'Independence Day',
    date(2021, 9, 6)    : 'Labor Day',
    date(2021, 10, 11)  : 'Columbus Day',
    date(2021, 11, 11)  : 'Veterans Day',
    date(2021, 11, 25)  : 'Thanksgiving',
    date(2021, 12, 24)  : 'Christmas'
}

solstice_and_equinox = {
    date(2021, 3, 21)  : 'Spring',
    date(2021, 6, 21)  : 'Summer',
    date(2021, 9, 23)  : 'Fall',
    date(2021, 12, 22) : 'Winter',
    date(2022, 3, 21)  : 'Spring',
}

months = {
    1  : 'January',
    2  : 'February',
    3  : 'March',
    4  : 'April',
    5  : 'May',
    6  : 'June',
    7  : 'July',
    8  : 'August',
    9  : 'September',
    10 : 'October',
    11 : 'November',
    12 : 'December'
}

fiscal_year_end = date(2021,7,31)

quarters = {
    date(2021, 1, 1)  : 1,
    date(2021, 4, 1)  : 2,
    date(2021, 7, 1)  : 3,
    date(2021, 10, 1) : 4,
    date(2022, 1, 1)  : 1
}

In [13]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

simulation_start_date = date(2020, 12, 31)
simulation_end_date = date(2021, 6, 30)
current_date = simulation_start_date

key = 365 #last key currently in db
dayNum = 0
quarter = 1
quarter_index = 0
season = 'Winter'
season_index = 0

fiscalYear = 2021
iday = 0
while(current_date != simulation_end_date):
    if (( (iday+1) % 20)==0):
        print("Working on Day ", iday + 1)
    current_date += datetime.timedelta(1)
    key += 1
    dayNum += 1

    weekend = False
    if current_date.weekday() >= 5:
        weekend = True
    
    if current_date.month == "1" and current_date.day == "1":
        dayNum = 1

    if current_date > fiscal_year_end:
        fiscalYear = 2020
    
    if current_date == list(solstice_and_equinox.keys())[season_index]:
        season = list(solstice_and_equinox.values())[season_index]
        season_index += 1

    if current_date == list(quarters.keys())[quarter_index]:
        quarter = list(quarters.values())[quarter_index]
        quarter_index += 1

    weekNum = current_date.isocalendar()[1]
    monthNum = current_date.month
    monthText = months[monthNum]
    isHoliday = current_date in holidays.keys()

    cur.execute("INSERT INTO DateDimension VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", 
        (key, current_date, current_date.day, dayNum, weekNum, monthNum, monthText, quarter, current_date.year, fiscalYear, isHoliday, weekend, season))

con.commit()
con.close()

### Add new 6 month data to existing Daily Fact Table 

In [14]:
#Store 4 data

con = lite.connect(r'Grocery.db')
cur = con.cursor()

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS TransactView;')
s2  = 'CREATE VIEW TransactView AS '
s2 += 'SELECT team4_6mo_transactions.*, COUNT(team4_6mo_transactions.SKU) AS numberSold, ROUND(SUM(sale_price), 2) AS saletotalSKU, DateKey '
s2 += 'FROM team4_6mo_transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = team4_6mo_transactions.date1 '
s2 += 'GROUP BY date1, team4_6mo_transactions.SKU;'

cur.execute(s2)

s3 = 'SELECT DateKey, TransactView.SKU, sale_price, numberSold, saletotalSKU, ProductKey, ROUND((BasePrice * numberSold), 2) ' 
s3 += 'as productBasePriceTotal, BasePrice '
s3 += 'FROM TransactView '
s3 += 'JOIN ProductDimension ON TransactView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY DateKey, TransactView.SKU'

results = cur.execute(s3)


count = 0

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[5]
    StoreKey = 4
    NumSoldToday = row[3]
    CostOfItemSold = row[6]
    SalesTotal = row[4]
    GrossProfit = round((SalesTotal - CostOfItemSold),2)
    SalePrice = row[2]
    BasePrice = row[7]
    
    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelSalesFact VALUES (?,?,?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumSoldToday, CostOfItemSold, SalesTotal, 
                                 GrossProfit, SalePrice, BasePrice))

print(count, " rows inserted")

con.commit()
con.close()

364906  rows inserted


In [15]:
#Store 5 data

con = lite.connect(r'Grocery.db')
cur = con.cursor()

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS TransactView;')
s2  = 'CREATE VIEW TransactView AS '
s2 += 'SELECT team5_6mo_transactions.*, COUNT(team5_6mo_transactions.SKU) AS numberSold, ROUND(SUM(sale_price), 2) AS saletotalSKU, DateKey '
s2 += 'FROM team5_6mo_transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = team5_6mo_transactions.date1 '
s2 += 'GROUP BY date1, team5_6mo_transactions.SKU;'

cur.execute(s2)

s3 = 'SELECT DateKey, TransactView.SKU, sale_price, numberSold, saletotalSKU, ProductKey, ROUND((BasePrice * numberSold), 2) ' 
s3 += 'as productBasePriceTotal, BasePrice '
s3 += 'FROM TransactView '
s3 += 'JOIN ProductDimension ON TransactView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY DateKey, TransactView.SKU'

results = cur.execute(s3)


count = 0

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[5]
    StoreKey = 5
    NumSoldToday = row[3]
    CostOfItemSold = row[6]
    SalesTotal = row[4]
    GrossProfit = round((SalesTotal - CostOfItemSold),2)
    SalePrice = row[2]
    BasePrice = row[7]
    
    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelSalesFact VALUES (?,?,?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumSoldToday, CostOfItemSold, SalesTotal, 
                                 GrossProfit, SalePrice, BasePrice))

print(count, " rows inserted")

con.commit()
con.close()

365391  rows inserted


In [16]:
#Store 6 data

con = lite.connect(r'Grocery.db')
cur = con.cursor()

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS TransactView;')
s2  = 'CREATE VIEW TransactView AS '
s2 += 'SELECT team6_6mo_transactions.*, COUNT(team6_6mo_transactions.SKU) AS numberSold, ROUND(SUM(sale_price), 2) AS saletotalSKU, DateKey '
s2 += 'FROM team6_6mo_transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = team6_6mo_transactions.date1 '
s2 += 'GROUP BY date1, team6_6mo_transactions.SKU;'

cur.execute(s2)

s3 = 'SELECT DateKey, TransactView.SKU, sale_price, numberSold, saletotalSKU, ProductKey, ROUND((BasePrice * numberSold), 2) ' 
s3 += 'as productBasePriceTotal, BasePrice '
s3 += 'FROM TransactView '
s3 += 'JOIN ProductDimension ON TransactView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY DateKey, TransactView.SKU'

results = cur.execute(s3)


count = 0

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[5]
    StoreKey = 6
    NumSoldToday = row[3]
    CostOfItemSold = row[6]
    SalesTotal = row[4]
    GrossProfit = round((SalesTotal - CostOfItemSold),2)
    SalePrice = row[2]
    BasePrice = row[7]
    
    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelSalesFact VALUES (?,?,?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumSoldToday, CostOfItemSold, SalesTotal, 
                                 GrossProfit, SalePrice, BasePrice))

print(count, " rows inserted")

con.commit()
con.close()

365778  rows inserted


In [17]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()
low = '2021-02-02'
high = '2021-02-08'
s = 'select * from DailyLevelSalesFact where DateKey = 370'
cur.execute(s)
rows = cur.fetchall()
